# Training the "big" model

The code above will load training data and train the so-called "big" model.

Let us first set up some parameters and load the train/test data.

In [1]:
import tensorflow as tf
from pndapetzim.data import load_datasets, LABEL_FILE_NAME, ORDER_FILE_NAME
from pndapetzim.models import CategoricalFeatureDescriptor

# Length of customer history to consider
seq_len = 20

# Relative weight of returning customer samples, an equivalent of oversampling.
returning_weight = 5.0

train, test, encodings = load_datasets(
    order_path='../data/' + ORDER_FILE_NAME,
    label_path='../data/' + LABEL_FILE_NAME,
    seq_len=seq_len,
    train_ratio=100,
    returning_weight=returning_weight,
)

# This dictionary holds the number of unique values and the target size of the embedding space (hyperparameters).
cat_features = {
    'restaurant_id': CategoricalFeatureDescriptor(
        vocab_size=encodings['restaurant_id'].vocab_size, embedding_size=15
    ),
    'city_id': CategoricalFeatureDescriptor(
        vocab_size=encodings['city_id'].vocab_size, embedding_size=12
    ),
    'payment_id': CategoricalFeatureDescriptor(
        vocab_size=encodings['payment_id'].vocab_size, embedding_size=3
    ),
    'platform_id': CategoricalFeatureDescriptor(
        vocab_size=encodings['platform_id'].vocab_size, embedding_size=4
    ),
    'transmission_id': CategoricalFeatureDescriptor(
        vocab_size=encodings['transmission_id'].vocab_size, embedding_size=4
    ),
}

Now we can build model...

In [2]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC, Recall
from tensorflow.keras.optimizers import Adam

from pndapetzim.models import build_large_model, CategoricalFeatureDescriptor

lossm = BinaryCrossentropy()
optimiser = Adam(learning_rate=0.01)

model = build_large_model(seq_len, cat_features)
aucm = AUC()
recallm = Recall()
metrics = ['accuracy', aucm, recallm]
model.compile(loss=lossm, optimizer=optimiser, metrics=metrics)
batch_size = 128

...and train it (takes some minutes):

In [3]:
model.fit(train.batch(batch_size).prefetch(10), epochs=1)

1899/1899 [==============================] - 864s 453ms/step - loss: 1.0346 - accuracy: 0.6929 - auc: 0.7828 - recall: 0.7407


Finally, let us evaluate the trained model on the test data:

In [5]:
loss, accuracy, auc, recall = model.evaluate(test.batch(batch_size))
print(f'loss: {loss}, accuracy: {accuracy}, AUC: {auc}, recall: {recallm.result()}')

19/19 [==============================] - 4s 144ms/step - loss: 0.9574 - accuracy: 0.7428 - auc: 0.8113 - recall: 0.7068
loss: 0.9574361443519592, accuracy: 0.7427983283996582, AUC: 0.8113062381744385, recall: 0.7067961096763611
